### Introduction to SQL for Data Science

SQL (Structured Query Language) is a powerful language used to manage and manipulate relational databases. Understanding SQL is crucial for:

*   **Data Extraction**: Retrieving specific datasets from large databases.
*   **Data Cleaning and Transformation**: Preprocessing data directly within the database before analysis.
*   **Feature Engineering**: Creating new features by combining or aggregating existing data.
*   **Understanding Data Structure**: Gaining insights into how data is organized.

We'll use an in-memory SQLite database for our examples, which is perfect for demonstration purposes as it doesn't require a separate database server setup.

In [1]:
import sqlite3
import pandas as pd

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

print("SQLite database created in memory.")

SQLite database created in memory.


### CRUD Operations

CRUD stands for Create, Read, Update, and Delete. These are the four basic functions of persistent storage. In SQL, these operations translate to:

*   **C**reate: `CREATE TABLE`, `INSERT INTO`
*   **R**ead: `SELECT`
*   **U**pdate: `UPDATE`
*   **D**elete: `DELETE FROM`

#### 1. CREATE TABLE

The `CREATE TABLE` statement is used to create a new table in a database. You define the table name and its columns, along with their data types and constraints (e.g., `PRIMARY KEY`, `NOT NULL`).

**Common SQL Data Types:**

*   `INTEGER`: Whole numbers.
*   `TEXT`: Character strings (like Python strings).
*   `REAL`: Floating-point numbers.
*   `BLOB`: Binary large objects (for storing raw binary data).
*   `DATE`, `DATETIME`: Date and time values.

In [2]:
# Create a table named 'Students'
cursor.execute('''
CREATE TABLE Students (
    student_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    age INTEGER,
    major TEXT
);
''')

# Create another table named 'Courses'
cursor.execute('''
CREATE TABLE Courses (
    course_id INTEGER PRIMARY KEY,
    course_name TEXT NOT NULL,
    credits INTEGER
);
''')

# Create a linking table for many-to-many relationship
cursor.execute('''
CREATE TABLE Enrollments (
    enrollment_id INTEGER PRIMARY KEY,
    student_id INTEGER,
    course_id INTEGER,
    grade TEXT,
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (course_id) REFERENCES Courses(course_id)
);
''')

conn.commit()
print("Tables 'Students', 'Courses', and 'Enrollments' created successfully.")

Tables 'Students', 'Courses', and 'Enrollments' created successfully.


#### 2. INSERT INTO

The `INSERT INTO` statement is used to add new rows (records) of data into a table.

In [3]:
# Insert data into the Students table
students_data = [
    (1, 'Alice', 'Smith', 20, 'Computer Science'),
    (2, 'Bob', 'Johnson', 22, 'Data Science'),
    (3, 'Charlie', 'Brown', 21, 'Mathematics'),
    (4, 'Diana', 'Prince', 23, 'Data Science'),
    (5, 'Eve', 'Adams', 19, 'Physics')
]
cursor.executemany("INSERT INTO Students VALUES (?, ?, ?, ?, ?)", students_data)

# Insert data into the Courses table
courses_data = [
    (101, 'Introduction to Python', 3),
    (102, 'Database Management', 4),
    (103, 'Machine Learning Basics', 3),
    (104, 'Calculus I', 4)
]
cursor.executemany("INSERT INTO Courses VALUES (?, ?, ?)", courses_data)

# Insert data into the Enrollments table
enrollments_data = [
    (1, 1, 101, 'A'),
    (2, 1, 102, 'B'),
    (3, 2, 102, 'A'),
    (4, 2, 103, 'A-'),
    (5, 3, 104, 'B+'),
    (6, 4, 102, 'B'),
    (7, 4, 103, 'A'),
    (8, 5, 101, 'C')
]
cursor.executemany("INSERT INTO Enrollments VALUES (?, ?, ?, ?)", enrollments_data)

conn.commit()
print("Data inserted into tables.")

Data inserted into tables.


#### 3. SELECT

`SELECT` is used to retrieve data from one or more tables. It's the most frequently used SQL command in data analysis.

*   `FROM`: Specifies the table(s) to retrieve data from.
*   `WHERE`: Filters rows based on specified conditions.
*   `ORDER BY`: Sorts the result set by one or more columns.
*   `LIMIT`: Restricts the number of rows returned.

In [4]:
# Select all columns and all rows from the Students table
print("\n--- All Students ---")
df_students = pd.read_sql_query("SELECT * FROM Students", conn)
display(df_students)

# Select specific columns from the Courses table
print("\n--- Course Names and Credits ---")
df_courses_info = pd.read_sql_query("SELECT course_name, credits FROM Courses", conn)
display(df_courses_info)

# Select students majoring in 'Data Science' (using WHERE clause)
print("\n--- Data Science Majors ---")
df_ds_majors = pd.read_sql_query("SELECT * FROM Students WHERE major = 'Data Science'", conn)
display(df_ds_majors)

# Select students older than 20, ordered by age (using WHERE and ORDER BY)
print("\n--- Students Older Than 20 (Ordered by Age) ---")
df_older_students = pd.read_sql_query("SELECT first_name, last_name, age FROM Students WHERE age > 20 ORDER BY age DESC", conn)
display(df_older_students)

# Select the top 2 students (using LIMIT)
print("\n--- Top 2 Students ---")
df_top2_students = pd.read_sql_query("SELECT * FROM Students LIMIT 2", conn)
display(df_top2_students)


--- All Students ---


,student_id,first_name,last_name,age,major
0,1,Alice,Smith,20,Computer Science
1,2,Bob,Johnson,22,Data Science
2,3,Charlie,Brown,21,Mathematics
3,4,Diana,Prince,23,Data Science
4,5,Eve,Adams,19,Physics



--- Course Names and Credits ---


,course_name,credits
0,Introduction to Python,3
1,Database Management,4
2,Machine Learning Basics,3
3,Calculus I,4



--- Data Science Majors ---


,student_id,first_name,last_name,age,major
0,2,Bob,Johnson,22,Data Science
1,4,Diana,Prince,23,Data Science



--- Students Older Than 20 (Ordered by Age) ---


,first_name,last_name,age
0,Diana,Prince,23
1,Bob,Johnson,22
2,Charlie,Brown,21



--- Top 2 Students ---


,student_id,first_name,last_name,age,major
0,1,Alice,Smith,20,Computer Science
1,2,Bob,Johnson,22,Data Science


#### 4. UPDATE

The `UPDATE` statement is used to modify existing records in a table. It's crucial to use a `WHERE` clause with `UPDATE` to specify which rows to change; otherwise, all rows in the table will be updated.

In [5]:
# Update the major of Alice Smith (student_id = 1) to 'Data Science'
cursor.execute("UPDATE Students SET major = 'Data Science' WHERE student_id = 1")
conn.commit()

print("\n--- Students table after update ---")
df_updated_students = pd.read_sql_query("SELECT * FROM Students", conn)
display(df_updated_students)


--- Students table after update ---


,student_id,first_name,last_name,age,major
0,1,Alice,Smith,20,Data Science
1,2,Bob,Johnson,22,Data Science
2,3,Charlie,Brown,21,Mathematics
3,4,Diana,Prince,23,Data Science
4,5,Eve,Adams,19,Physics


#### 5. DELETE

The `DELETE FROM` statement is used to delete existing records from a table. Like `UPDATE`, always use a `WHERE` clause to specify which rows to delete; otherwise, all records in the table will be removed.

In [6]:
# Delete the student 'Eve Adams' (student_id = 5)
cursor.execute("DELETE FROM Students WHERE student_id = 5")
conn.commit()

print("\n--- Students table after deletion ---")
df_deleted_students = pd.read_sql_query("SELECT * FROM Students", conn)
display(df_deleted_students)

# Note: Deleting a student here might leave orphaned enrollment records if foreign key constraints are not set up with CASCADE DELETE, or if SQLite is in legacy mode. For simplicity, we are not enforcing CASCADE DELETE in this example.
# You might want to delete related enrollment records as well:
# cursor.execute("DELETE FROM Enrollments WHERE student_id = 5")
# conn.commit()


--- Students table after deletion ---


,student_id,first_name,last_name,age,major
0,1,Alice,Smith,20,Data Science
1,2,Bob,Johnson,22,Data Science
2,3,Charlie,Brown,21,Mathematics
3,4,Diana,Prince,23,Data Science


### SQL Joins

Joins are used to combine rows from two or more tables based on a related column between them. This is a fundamental operation in relational databases, allowing you to retrieve a complete picture of your data from various sources.

#### Types of Joins:

1.  **`INNER JOIN`**: Returns rows when there is a match in *both* tables.
2.  **`LEFT JOIN` (or `LEFT OUTER JOIN`)**: Returns all rows from the left table, and the matching rows from the right table. If there is no match, `NULL` is used for columns from the right table.
3.  **`RIGHT JOIN` (or `RIGHT OUTER JOIN`)**: Returns all rows from the right table, and the matching rows from the left table. If there is no match, `NULL` is used for columns from the left table.
4.  **`FULL OUTER JOIN`**: Returns all rows when there is a match in *either* the left or the right table. If there is no match, `NULL` is used for non-matching columns.

Let's demonstrate each with our `Students`, `Courses`, and `Enrollments` tables.

#### 1. INNER JOIN

`INNER JOIN` returns only the rows that have matching values in both tables. It's the most common type of join and is often implicitly assumed if you just use `JOIN`.

In [7]:
# INNER JOIN: Get students and their enrolled courses (only where a student is actually enrolled)
print("\n--- INNER JOIN: Students with their enrolled courses ---")
df_inner_join = pd.read_sql_query(
    """SELECT
        S.first_name, S.last_name, C.course_name, E.grade
    FROM
        Students AS S
    INNER JOIN
        Enrollments AS E ON S.student_id = E.student_id
    INNER JOIN
        Courses AS C ON E.course_id = C.course_id;""", conn
)
display(df_inner_join)


--- INNER JOIN: Students with their enrolled courses ---


,first_name,last_name,course_name,grade
0,Alice,Smith,Introduction to Python,A
1,Alice,Smith,Database Management,B
2,Bob,Johnson,Database Management,A
3,Bob,Johnson,Machine Learning Basics,A-
4,Charlie,Brown,Calculus I,B+
5,Diana,Prince,Database Management,B
6,Diana,Prince,Machine Learning Basics,A


#### 2. LEFT JOIN

`LEFT JOIN` (or `LEFT OUTER JOIN`) returns all rows from the 'left' table, and the matching rows from the 'right' table. If there's no match in the right table, `NULL` values are returned for the right table's columns. This is useful when you want to keep all records from one table, even if there are no corresponding records in another.

In [8]:
# LEFT JOIN: Get all students and their enrolled courses. Students without enrollments will still appear with NULLs for course info.
print("\n--- LEFT JOIN: All Students and their enrollments (if any) ---")
df_left_join = pd.read_sql_query(
    """SELECT
        S.first_name, S.last_name, C.course_name, E.grade
    FROM
        Students AS S
    LEFT JOIN
        Enrollments AS E ON S.student_id = E.student_id
    LEFT JOIN
        Courses AS C ON E.course_id = C.course_id;""", conn
)
display(df_left_join)


--- LEFT JOIN: All Students and their enrollments (if any) ---


,first_name,last_name,course_name,grade
0,Alice,Smith,Introduction to Python,A
1,Alice,Smith,Database Management,B
2,Bob,Johnson,Database Management,A
3,Bob,Johnson,Machine Learning Basics,A-
4,Charlie,Brown,Calculus I,B+
5,Diana,Prince,Database Management,B
6,Diana,Prince,Machine Learning Basics,A


#### 3. RIGHT JOIN

`RIGHT JOIN` (or `RIGHT OUTER JOIN`) returns all rows from the 'right' table, and the matching rows from the 'left' table. If there's no match in the left table, `NULL` values are returned for the left table's columns.

**Note:** SQLite does not directly support `RIGHT JOIN`. However, you can achieve the same result by swapping the tables and using a `LEFT JOIN`.

In [9]:
# Simulating RIGHT JOIN with LEFT JOIN: Get all courses and their enrollments (if any). Courses without enrollments will still appear with NULLs for student info.
print("\n--- RIGHT JOIN (Simulated): All Courses and their enrollments (if any) ---")
df_right_join_simulated = pd.read_sql_query(
    """SELECT
        C.course_name, S.first_name, S.last_name, E.grade
    FROM
        Courses AS C
    LEFT JOIN
        Enrollments AS E ON C.course_id = E.course_id
    LEFT JOIN
        Students AS S ON E.student_id = S.student_id;""", conn
)
display(df_right_join_simulated)


--- RIGHT JOIN (Simulated): All Courses and their enrollments (if any) ---


,course_name,first_name,last_name,grade
0,Introduction to Python,Alice,Smith,A
1,Introduction to Python,None,None,C
2,Database Management,Alice,Smith,B
3,Database Management,Bob,Johnson,A
4,Database Management,Diana,Prince,B
5,Machine Learning Basics,Bob,Johnson,A-
6,Machine Learning Basics,Diana,Prince,A
7,Calculus I,Charlie,Brown,B+


#### 4. FULL OUTER JOIN

`FULL OUTER JOIN` returns all rows when there is a match in one of the tables. If there's no match, `NULL` is used for columns from the non-matching side. It combines the results of both `LEFT JOIN` and `RIGHT JOIN`.

**Note:** SQLite also does not directly support `FULL OUTER JOIN`. You can simulate it by combining a `LEFT JOIN` and a `RIGHT JOIN` (simulated as a `LEFT JOIN` with swapped tables) using `UNION ALL` and filtering out duplicates or combining `NULL` values appropriately. A common way is using `LEFT JOIN` and then `UNION ALL` with the non-matching part of the 'right' table.

In [10]:
# Simulating FULL OUTER JOIN in SQLite: Get all students and all courses, showing enrollments where they exist, and NULLs where there are no matches.
print("\n--- FULL OUTER JOIN (Simulated): All Students, All Courses, and their enrollments ---")
df_full_outer_join_simulated = pd.read_sql_query(
    """SELECT
        S.first_name, S.last_name, C.course_name, E.grade
    FROM
        Students AS S
    LEFT JOIN
        Enrollments AS E ON S.student_id = E.student_id
    LEFT JOIN
        Courses AS C ON E.course_id = C.course_id
    UNION ALL
    SELECT
        S.first_name, S.last_name, C.course_name, E.grade
    FROM
        Courses AS C
    LEFT JOIN
        Enrollments AS E ON C.course_id = E.course_id
    LEFT JOIN
        Students AS S ON E.student_id = S.student_id
    WHERE
        S.student_id IS NULL; -- Select only rows from Courses that didn't have a match in the first LEFT JOIN
    """, conn
)
display(df_full_outer_join_simulated)


--- FULL OUTER JOIN (Simulated): All Students, All Courses, and their enrollments ---


,first_name,last_name,course_name,grade
0,Alice,Smith,Introduction to Python,A
1,Alice,Smith,Database Management,B
2,Bob,Johnson,Database Management,A
3,Bob,Johnson,Machine Learning Basics,A-
4,Charlie,Brown,Calculus I,B+
5,Diana,Prince,Database Management,B
6,Diana,Prince,Machine Learning Basics,A
7,None,None,Introduction to Python,C


### SQL Aggregate Functions

Aggregate functions perform calculations on a set of rows and return a single summary value. They are incredibly useful for data summarization and analysis.

#### Common Aggregate Functions:

*   **`COUNT()`**: Returns the number of rows that match a specified criterion.
*   **`SUM()`**: Calculates the sum of a set of values.
*   **`AVG()`**: Calculates the average of a set of values.
*   **`MIN()`**: Returns the smallest value in a set.
*   **`MAX()`**: Returns the largest value in a set.
*   **`GROUP BY`**: Often used with aggregate functions to group the result-set by one or more columns.

In [11]:
# COUNT: Get the total number of students
print("\n--- Total Number of Students ---")
df_count_students = pd.read_sql_query("SELECT COUNT(*) AS total_students FROM Students;", conn)
display(df_count_students)

# AVG: Calculate the average age of students
print("\n--- Average Student Age ---")
df_avg_age = pd.read_sql_query("SELECT AVG(age) AS average_age FROM Students;", conn)
display(df_avg_age)

# SUM: Calculate the total credits for all courses
print("\n--- Total Course Credits ---")
df_sum_credits = pd.read_sql_query("SELECT SUM(credits) AS total_credits FROM Courses;", conn)
display(df_sum_credits)

# MIN and MAX: Get the minimum and maximum age of students
print("\n--- Min and Max Student Age ---")
df_min_max_age = pd.read_sql_query("SELECT MIN(age) AS min_age, MAX(age) AS max_age FROM Students;", conn)
display(df_min_max_age)

# GROUP BY with COUNT: Count students by major
print("\n--- Students Count by Major ---")
df_students_by_major = pd.read_sql_query("SELECT major, COUNT(*) AS num_students FROM Students GROUP BY major;", conn)
display(df_students_by_major)

# GROUP BY with AVG: Average age by major
print("\n--- Average Age by Major ---")
df_avg_age_by_major = pd.read_sql_query("SELECT major, AVG(age) AS average_age FROM Students GROUP BY major;", conn)
display(df_avg_age_by_major)

# GROUP BY with SUM and JOIN: Total credits enrolled by each student
print("\n--- Total Credits Enrolled by Each Student ---")
df_student_credits = pd.read_sql_query(
    """SELECT
        S.first_name, S.last_name, SUM(C.credits) AS total_enrolled_credits
    FROM
        Students AS S
    INNER JOIN
        Enrollments AS E ON S.student_id = E.student_id
    INNER JOIN
        Courses AS C ON E.course_id = C.course_id
    GROUP BY
        S.student_id, S.first_name, S.last_name
    ORDER BY
        total_enrolled_credits DESC;""", conn
)
display(df_student_credits)


--- Total Number of Students ---


,total_students
0,4



--- Average Student Age ---


,average_age
0,21.5



--- Total Course Credits ---


...

,major,num_students
0,Data Science,3
1,Mathematics,1



--- Average Age by Major ---


,major,average_age
0,Data Science,21.666667
1,Mathematics,21.000000



--- Total Credits Enrolled by Each Student ---


,first_name,last_name,total_enrolled_credits
0,Alice,Smith,7
1,Bob,Johnson,7
2,Diana,Prince,7
3,Charlie,Brown,4
